   > # 1. 기존 다람쥐 버스의 경로 분석 
   - 기존다람쥐 버스의 승/하차인원과 전체 정류장의 승/하차인원(유동인구) 시각화

In [1]:
import sys 
import pandas as pd

sys.path.append('./EDA')
sys.path.append('./Modeling')
sys.path.append('./Data/csv')

from EDA_STEP_1 import *
from EDA.folium import *

# 모든 컬럼을 출력하도록 display 옵션 설정
pd.set_option('display.max_columns', 35)

### 1-1 기존 다람쥐 버스 노선 및 승하차 인원 데이터 변형

In [41]:
# gmm 다람쥐 csv 가져오기
gmm_daram_df = pd.read_csv('./Data/csv/gmm_daram.csv', encoding='UTF-8')
gmm_daram_df=gmm_daram_df[['X좌표','Y좌표','NODE_ID', 'gmm_cluster']]
gmm_daram_df

# 다람쥐 95개 정류장 csv 가져오기
daram_df = pd.read_csv('./Data/csv/다람쥐95개정류장.csv', encoding='cp949')

# 노드별 정류장 순서 csv 가져오기
bus_route_info = pd.read_csv('./Data/csv/bus_route_info.csv')

# 95개 다람쥐 버스 정류장에 gmm군집결과를 labeling
gmm_daram_result_df = pd.merge(daram_df,gmm_daram_df, left_on='NODE_ID' ,right_on='NODE_ID',how='left')

# 컬럼 이름 변경
gmm_daram_result_df.rename(columns={"X좌표_x":"X좌표","Y좌표_x":"Y좌표"},inplace=True)

# 히트맵을 위한 승+하차 인원 컬럼 생성
gmm_daram_result_df["ride_alight_sum"]=gmm_daram_result_df["RIDE_SUM_6_10"]+gmm_daram_result_df["ALIGHT_SUM_6_10"]

# 병합을 위한 형태 변환
gmm_daram_result_df.loc[:,"노선번호"] = gmm_daram_result_df.loc[:,"노선번호"].astype("str")

# 버스 정류장 순서를 위해 
gmm_daram_result_df = pd.merge(left = bus_route_info, right = gmm_daram_result_df, left_on = ["NODE_ID",'노선명'],right_on=["NODE_ID",'노선번호'],how="inner")

# 컬럼 이름 변경
gmm_daram_result_df.rename(columns={"X좌표_x":"X좌표","Y좌표_x":"Y좌표"},inplace=True)

# 노선별 순번으로 정렬
gmm_daram_result_df = gmm_daram_result_df.sort_values(by=["노선명","순번"])

/var/folders/dj/074406ws2d7f7vkzzjkhcfp80000gn/T/ipykernel_44552/1874648059.py:25: FutureWarning: Passing 'suffixes' which cause duplicate columns {'X좌표_y', 'Y좌표_y'} in the result is deprecated and will raise a MergeError in a future version.
  gmm_daram_result_df = pd.merge(left = bus_route_info, right = gmm_daram_result_df, left_on = ["NODE_ID",'노선명'],right_on=["NODE_ID",'노선번호'],how="inner")


In [38]:
# 1315개 군집 정류장의 승하차인원 데이터 호출
cosine_similarity_daram = pd.read_csv("./Data/csv/cosine_similarity_daram.csv")
# 컬럼 이름 통일을 위해 변경
cosine_similarity_daram.rename(columns={"Label":'gmm_cluster'},inplace=True)
# 클래스 값을 덧붙이기위해 정렬
gmm_daram = gmm_daram.sort_values(by="NODE_ID")
cosine_similarity_daram = cosine_similarity_daram.sort_values(by="NODE_ID")
# 클래스 컬럼 붙이기
cosine_similarity_daram["gmm_cluster"] = gmm_daram["gmm_cluster"]

In [61]:
cosine_similarity_daram

,Unnamed: 0.1,index,Unnamed: 0,NODE_ID,정류소명,X좌표,Y좌표,법정동코드,법정동_구,법정동,academy_cnt,kindergarten_cnt,mart_cnt,restaurant_cnt,school_cnt,university_cnt,subway_cnt,tour_cnt,cafe_cnt,hospital_cnt,culture_cnt,univ_hospital_cnt,public_office_cnt,employee_cnt,alone_ratio,emp_corp_ratio,population_15to64,RIDE_SUM_6_10,ALIGHT_SUM_6_10,Label,gmm_cluster
0,35,35,35,100000037,국민은행세검정지점,126.960888,37.604663,1111018600,종로구,신영동,31,3,1,70,1,0,0,0,18,3,9,0,1,1255,0.123238,4.297945,2188,80082,39699,0,0
1,36,36,36,100000038,세검정초등학교,126.961220,37.602502,1111018600,종로구,신영동,31,5,0,66,2,0,0,1,17,3,10,0,2,1255,0.123238,4.297945,2188,254790,99879,0,0
2,37,37,37,100000039,세검정.상명대,126.958722,37.600733,1111018500,종로구,홍지동,25,4,0,53,4,1,0,0,22,3,13,0,1,1255,0.123238,4.297945,2188,19894,46761,0,0
3,38,38,38,100000040,상명대입구.석파랑,126.958570,37.599413,1111018500,종로구,홍지동,10,4,0,45,3,1,0,1,20,4,9,0,1,1255,0.123238,4.297945,2188,48158,19098,0,0
4,39,39,39,100000041,하림각,126.961707,37.598133,1111018400,종로구,부암동,13,3,0,41,0,0,0,2,19,4,6,0,1,1255,0.123238,4.297945,2188,154700,45650,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1310,12401,12496,12496,124900086,천호동로데오거리,127.127198,37.540565,1174010900,강동구,천호동,58,8,3,642,0,0,2,8,114,139,3,0,3,29406,0.181504,3.280455,62587,1220,73802,0,10
1311,12402,12497,12497,124900087,천호우체국,127.125362,37.540860,1174010900,강동구,천호동,62,7,3,588,0,0,2,8,109,139,3,0,4,29406,0.181504,3.280455,62587,18355,3539,0,10
1312,12403,12498,12498,124900088,천호동로데오거리,127.127189,37.540362,1174010900,강동구,천호동,59,7,3,664,0,0,2,8,122,139,3,0,3,29406,0.181504,3.280455,62587,6823,5343,0,10
1313,12404,12499,12499,124900089,암사종합시장,127.128250,37.549699,1174010900,강동구,천호동,97,29,0,385,3,0,1,0,64,63,0,0,4,29406,0.181504,3.280455,62587,36479,1697,0,10


### 1-2 전체 정류장 승하차 데이터 변형

In [3]:
# 승차인원_월별_시간별 데이터 불러오기
bus_month_hour = pd.read_csv("./Data/csv/승차인원_월별_시간별.csv")

bus_route_info["NODE_ID"] = bus_route_info["NODE_ID"].astype("float64")

# 모든 버스 정류장 6~9시 승하차 인원수
bus_month_hour["6_9시_승차"] = bus_month_hour["6시승차총승객수"]+bus_month_hour["7시승차총승객수"]+bus_month_hour["8시승차총승객수"]+bus_month_hour["9시승차총승객수"]
bus_month_hour["6_9시_하차"] = bus_month_hour["6시하차총승객수"]+bus_month_hour["7시하차총승객수"]+bus_month_hour["8시하차총승객수"]+bus_month_hour["9시하차총승객수"]

# 정류장 정렬
bus_month_hour_sort = bus_month_hour.sort_values(by=["노선번호","6_9시_승차"],ascending=False)

# 병합을 위한 컬럼 "NODE_ID"를 위해 병합
station_xy = pd.read_csv("./Data/csv/버스_정류장_xy.csv")

a = pd.merge(left = bus_month_hour_sort, right = station_xy[["NODE_ID","X좌표","Y좌표"]], left_on= "표준버스정류장ID",right_on="NODE_ID",how="left")

bus_info_xy = pd.concat([a.loc[:,"노선번호":"역명"],a.loc[:,"6시승차총승객수":"9시하차총승객수"],a.loc[:,"6_9시_승차":]],axis=1)

bus_info_xy_headmap_on_off = bus_info_xy.dropna()
bus_info_xy_headmap_on_off_sum= bus_info_xy_headmap_on_off.groupby(by="표준버스정류장ID")["6_9시_승차","6_9시_하차"].sum().reset_index()
bus_info_xy_headmap_on_off = bus_info_xy_headmap_on_off.merge(bus_info_xy_headmap_on_off_sum, on="표준버스정류장ID",suffixes=('', '_sum'))
# 동일좌표 정류장 승하차별 합 컬럼생성
bus_info_xy_headmap_on_off["승하차총합"] = bus_info_xy_headmap_on_off["6_9시_승차_sum"]+bus_info_xy_headmap_on_off["6_9시_하차_sum"]

# 노선별 버스 순번 데이터 합치기
total_df = pd.merge(left = bus_info_xy_headmap_on_off, right = bus_route_info[['노선명', '순번', 'NODE_ID']], left_on= ["노선번호","표준버스정류장ID"],right_on=["노선명","NODE_ID"],how="inner")
total_df = total_df.sort_values(by=["노선번호","순번"])

total_df.drop_duplicates(inplace=True)

/var/folders/dj/074406ws2d7f7vkzzjkhcfp80000gn/T/ipykernel_44552/2691608786.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  bus_month_hour = pd.read_csv("./Data/csv/승차인원_월별_시간별.csv")
/var/folders/dj/074406ws2d7f7vkzzjkhcfp80000gn/T/ipykernel_44552/2691608786.py:21: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  bus_info_xy_headmap_on_off_sum= bus_info_xy_headmap_on_off.groupby(by="표준버스정류장ID")["6_9시_승차","6_9시_하차"].sum().reset_index()


In [4]:
# 유사도로 군집된 1315개의 정류장 데이터 호출
gmm_daram = pd.read_csv("./Data/csv/gmm_daram.csv")
gmm_daram

,Unnamed: 0,pca1,pca2,pca3,pca4,pca5,pca6,gmm_cluster,index,X좌표,Y좌표,NODE_ID
0,1119,-0.617303,-1.459993,0.389597,3.754457,-1.562935,0.086068,0,8691,126.918884,37.521891,118000076
1,1101,0.378930,-1.079291,-0.046722,3.408468,-1.211176,-0.692579,0,9019,126.930759,37.525932,118000613
2,1164,-0.069205,1.084566,1.101227,2.018559,-1.579968,-0.601973,0,8433,126.893399,37.476205,117900029
3,1096,1.309512,-1.127871,1.382548,3.502019,-0.978033,0.237145,0,8666,126.924904,37.530782,118000051
4,1094,2.508437,-0.587880,0.831179,3.652004,-0.739060,0.164253,0,8799,126.920853,37.527967,118000196
...,...,...,...,...,...,...,...,...,...,...,...,...
1310,693,5.035301,-1.272405,3.190015,-3.294807,-0.199899,-2.680025,10,2806,127.000648,37.593218,107000140
1311,1009,1.397033,-1.372169,0.040843,0.195963,0.336420,0.366396,10,9075,126.897552,37.518672,118900051
1312,695,5.198289,-1.243730,4.051635,-2.473887,0.027647,-0.792429,10,2803,126.995944,37.593745,107000137
1313,684,4.785935,-1.268199,3.099309,-3.186075,-0.157146,-2.647045,10,2807,127.001531,37.593272,107000141


## 1-3 folium으로 시각화

- 기존 다람쥐 버스의 경로는 어떻게 선정했는지 분석하기 위해 folium으로 기존 다람쥐 버스의 승하차 인원을 "원의 크기"로, 유동인구를 확인하기 위해 모든 정류장의 승/하차인원의 합을 "히트맵"으로 나타냈다. 
- 초록색은 승차인원, 빨간색은 하차인원을 나타낸다.

In [20]:
folium_bus(total_df,"승하차총합",None,gmm_daram)

In [5]:
folium_bus(None,None,gmm_daram_result_df,gmm_daram_result_df,"midnight")

## 1-4 각 다람쥐 노선 분석
- 각 노선은 밀도가 높은 부분(승/하차가 높은 곳)을 경유
- 기점 근처는 하차보다 승차인원이 많은 경향을 보임
- 지하철역이 있는 종점에서는 승차보다 하차 인원이 많은 경향을 보임

<img src= "./Data/img/경유지분석1.png" width="500" height="450"/>
<img src= "./Data/img/경유지분석2.png" width="500" height="450"/>

# <분석결과>
>다람쥐 버스 노선 경유지 선정기준
- 기점, 종점 -> 승차, 하차 승객수(유동인구)가 많은 지역
- 경유지 -> 승하차 합계가 많은 곳
- 유사도는 경로 선정 시 참고사항임.

   > # 2. 기존 다람쥐 경로분석을 통한 신설 노선 선정
   - "특정 파일"의 기준을 바탕으로 1315개의 유사도 군집에서 걸러진 정류장들을 사용하여 신설 다람쥐 버스 노선의 기점,종점의 클래스를 정하였다.
   - 기점과 종점은 1. 마곡아파트-디지털미디어시티역 2. 명일역-강동구청역 3. 개포동-양재역으로 선정하였다.

<img src= "./Data/img/노선의최종경로선정1.png" width="500" height="450"/>

## 2-1 신설노선 경로 선정 과정
- 선정된 기점과 종점을 잇는 경로는 2단계의 과정을 거쳐 선정했다.

### 1단계
- 유사도 결과를 기반으로 승차와 하차가 많은 정류장을 기점과 종점으로 설정
- 노선방향은 승차인원이 많은 정류장에서 하차가 많은 정류장 방향으로 선택하였다. 
- 해당 경로의 승/하차인원은 기존에 존재하는 버스 노선의 승/하차인원을 사용하여 분석하였다.
- 기점과 종점을 잇는 경로는 기존 버스 노선들을 활용하였다.

In [59]:
folium_bus(None,None,cosine_similarity_daram)

KeyError: '노선번호'

## 재차인원 분석

In [6]:
강서_node_1 = bus_재차인원(total_df,"강서05",5,28)
강서_node_1

,노선번호,표준버스정류장ID,순번,역명,6시승차총승객수,6시하차총승객수,6시승하차_sub,7시승차총승객수,7시하차총승객수,7시승하차_sub,8시승차총승객수,8시하차총승객수,8시승하차_sub,9시승차총승객수,9시하차총승객수,9시승하차_sub,6시재차인원,7시재차인원,8시재차인원,9시재차인원,X좌표,Y좌표
0,강서05,115900134,5,수명산파크7단지상가(00005),216,12,204,695,27,668,559,26,533,382,17,365,204,668,533,365,126.822235,37.547383
1,강서05,115900132,6,수명산파크7단지정문(00006),517,2,515,955,27,928,1156,19,1137,884,15,869,719,1596,1670,1234,126.824277,37.547851
2,강서05,115900131,7,수명중.고등학교(00007),422,5,417,902,37,865,554,54,500,317,22,295,1136,2461,2170,1529,126.824169,37.550068
3,강서05,115900194,8,마곡수명산파크5단지.6단지(00008),552,27,525,1671,22,1649,1413,126,1287,817,75,742,1661,4110,3457,2271,126.825397,37.551408
4,강서05,115900129,9,수명산파크상가(00009),174,6,168,927,48,879,659,70,589,514,50,464,1829,4989,4046,2735,126.826063,37.553295
5,강서05,115900127,10,수명산파크5단지아파트(00010),575,8,567,1334,78,1256,1582,97,1485,774,39,735,2396,6245,5531,3470,126.828212,37.553394
6,강서05,115900126,11,수명산파크1단지아파트앞(00011),653,12,641,1023,69,954,1343,43,1300,860,35,825,3037,7199,6831,4295,126.829739,37.553258
7,강서05,115900125,12,수명산근린공원(00012),323,50,273,618,201,417,947,89,858,626,16,610,3310,7616,7689,4905,126.830922,37.551582
8,강서05,115900193,13,발산초등학교(00013),44,38,6,152,742,-590,121,181,-60,121,73,48,3316,7026,7629,4953,126.831836,37.549627
9,강서05,115900124,14,화곡중.고등학교(00014),95,508,-413,250,1193,-943,243,1115,-872,262,849,-587,2903,6083,6757,4366,126.833124,37.547876


In [7]:
강서_node_2 = bus_재차인원(total_df,"8762",8)
강서_node_2

,노선번호,표준버스정류장ID,순번,역명,6시승차총승객수,6시하차총승객수,6시승하차_sub,7시승차총승객수,7시하차총승객수,7시승하차_sub,8시승차총승객수,8시하차총승객수,8시승하차_sub,9시승차총승객수,9시하차총승객수,9시승하차_sub,6시재차인원,7시재차인원,8시재차인원,9시재차인원,X좌표,Y좌표
0,8762,115000039,9,가양역.마포중고등학교(00009),35,1,34,1041,2,1039,2041,8,2033,957,5,952,34,1039,2033,952,126.854204,37.558928
1,8762,113000180,10,월드컵파크7단지(00010),4,9,-5,28,140,-112,41,393,-352,22,193,-171,29,927,1681,781,126.878420,37.581150
2,8762,113000197,11,상암고등학교(00011),13,11,2,18,219,-201,11,739,-728,19,327,-308,31,726,953,473,126.884046,37.581780
3,8762,113000195,12,월드컵파크5단지(00012),0,5,-5,29,186,-157,53,643,-590,43,315,-272,26,569,363,201,126.886518,37.580333
4,8762,111001114,13,MBC(00013),4,5,-1,79,96,-17,150,240,-90,107,193,-86,25,552,273,115,126.890895,37.581652
5,8762,113000452,14,SBS.YTN(00014),0,2,-2,4,29,-25,11,192,-181,21,167,-146,23,527,92,-31,126.894029,37.579627
6,8762,113000450,15,공항철도디지털미디어시티역(00015),0,1,-1,1,126,-125,7,197,-190,8,145,-137,22,402,-98,-168,126.898088,37.576944


In [8]:
강서_node_3 = bus_재차인원(total_df,"8762",None,8)
강서_node_3

,노선번호,표준버스정류장ID,순번,역명,6시승차총승객수,6시하차총승객수,6시승하차_sub,7시승차총승객수,7시하차총승객수,7시승하차_sub,8시승차총승객수,8시하차총승객수,8시승하차_sub,9시승차총승객수,9시하차총승객수,9시승하차_sub,6시재차인원,7시재차인원,8시재차인원,9시재차인원,X좌표,Y좌표
0,8762,113000202,1,디지털미디어시티역2번출구(00001),170,1,169,1114,3,1111,1930,4,1926,869,7,862,169,1111,1926,862,126.899674,37.575208
1,8762,113000073,2,DMC상암센트럴파크(00002),6,2,4,54,27,27,12,49,-37,10,23,-13,173,1138,1889,849,126.896201,37.575044
2,8762,113000076,3,상암초등학교(00003),12,26,-14,41,85,-44,40,234,-194,7,86,-79,159,1094,1695,770,126.893493,37.576634
3,8762,113000192,4,상암DMC홍보관.YTN(00004),6,36,-30,14,169,-155,27,553,-526,3,248,-245,129,939,1169,525,126.891323,37.577985
4,8762,113000204,5,누리꿈스퀘어.MBC(00005),6,19,-13,42,280,-238,27,701,-674,10,274,-264,116,701,495,261,126.889637,37.578988
5,8762,113000194,6,월드컵파크5단지.상암중고등학교입구(00006),4,21,-17,103,223,-120,110,569,-459,48,305,-257,99,581,36,4,126.886335,37.580921
6,8762,113000181,7,월드컵파크7단지(00007),128,5,123,749,21,728,501,16,485,179,10,169,222,1309,521,173,126.878963,37.581876
7,8762,115000040,8,가양역.마포중고등학교(00008),0,142,-142,32,885,-853,143,743,-600,22,422,-400,80,456,-79,-227,126.854194,37.559559


In [ ]:
명일_node_3